In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from modules.KNN import *
from modules.DTW import *
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import *
from modules.ROC import *

In [ ]:
path = 'Assets/Isolated Digits'
data_class = ['2', '3', '4', '5', 'z']

C = []; D = []
for i in data_class:
	C.append(read_digit(path, i, 'train')[0])
	D.append(read_digit(path, i, 'dev')[0])

In [ ]:
C_y = []
D_y = []

for i in range(5):
	C_y.append(i * np.ones(len(C[i])))
	D_y.append(i * np.ones(len(D[i])))

In [ ]:
Cn = []
Dn = []
for c in range(5):
	C_new = np.array(C[c][0])
	D_new = np.array(D[c][0])
	for i in range(1,len(C[c])):
		C_new = np.vstack((C_new, np.array(C[c][i])))
	for i in range(1,len(D[c])):
		D_new = np.vstack((D_new, np.array(D[c][i])))
	Cn.append(C_new)
	Dn.append(D_new)

In [ ]:
Yn = []
Yn_d = []

for i in range(5):
	Yn.append(i * np.ones(len(Cn[i])))
	Yn_d.append(i * np.ones(len(Dn[i])))

Yn = np.concatenate(Yn, axis=0)
Yn_d = np.concatenate(Yn_d, axis=0)

In [ ]:
X = np.concatenate(Cn, axis=0)
X_d = np.concatenate(Dn, axis=0)
Y = np.concatenate(C_y, axis=0)
Y_d = np.concatenate(D_y, axis=0)

In [ ]:
pca = False
lda = False
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	pc = 15
	μ, Σ = stats(X)
	eigval, Q = PCA(Σ, pc)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 15
	μ, Σ = stats(X)
	μk = []
	for c in range(5):
		μk.append(stats(X[Yn == c])[0])
	μk = np.array(μk)
	v = LDA(X,Yn,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
if pca or lda:
	cn = 0
	dn = 0
	C_new = []
	D_new = []
	for c in range(5):
		t = []
		d = []
		for i in range(len(C[c])):
			t.append(X[cn:cn+len(C[c][i])])
			cn += len(C[c][i])
		for i in range(len(D[c])):
			d.append(X_d[dn:dn+len(D[c][i])])
			dn += len(D[c][i])
		D_new.append(d)
		C_new.append(t)
	D = D_new
	C = C_new

In [ ]:
P = []
for cl in range(5):
	p = []
	for dev in D[cl]:
		cost = []
		for c in range(5):
			for train in C[c]:
				cost.append([DTW(dev, train)[-1,-1], c])
		cost = np.array(cost)
		idx = cost[:,0].argsort()
		cost = cost[idx]
		p.append(cost[:, 1])
	P.append(p)

In [ ]:
k = 10
Q = []
score = []
for c in range(5):
	q = []
	s = []
	for j in range(len(D[c])):
		data = P[c][j][:k]
		q.append(statistics.mode(data))
		s.append(np.array([(data==0).sum()/k, (data==1).sum()/k, (data==2).sum()/k, (data==3).sum()/k, (data==4).sum()/k]))
	Q.append(q)
	score.append(s)
y_pred = np.concatenate(Q)
score = np.concatenate(score)


In [ ]:
cm = confusion_matrix(Y_d, y_pred)
accuracy = accuracy_score(Y_d,y_pred)
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = data_class, yticklabels = data_class)
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR = 0; FPR = 0
for c in range(5):
	tpr, fpr = ROC(label_binarize(Y_d, classes=[0,1,2,3,4])[:,c], score[:,c])
	TPR += tpr/5
	FPR += fpr/5
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")